In [568]:
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random

In [569]:
path_folder_data = 'data/ОПК трудоустройство'

In [570]:
# создаем словарь верхнего уровня для каждого поо
high_level_dct = {}
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])

In [571]:
for file in os.listdir(path_folder_data):
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        print(name_file)
        df_form1 = pd.read_excel(f'{path_folder_data}/{file}', skiprows=8, dtype=str)
        # создаем множество колонок наличие которых мы проверяем
        check_cols = {'01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17',
                      '18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36',
                     '37','38','39','40','41','42','43','44','45','46','47','48','49','50','51','52','53','54','55',
                     '56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','71','72','73','74',
                     '75','76','77','78','79','80'}
        if not check_cols.issubset(set(df_form1.columns)):
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'Проверьте заголовок таблицы в файле.Строка с номерами колонок (01,02,03 и т.д. как в исходной форме)\n должна находиться на 5 строке! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue
    

        df_form1 = df_form1[df_form1['04'] != '03'] # фильтруем строки с проверкой

        df_form1 = df_form1.loc[:,'01':'78'] # отсекаем возможную первую колонку и колонки с примечаниями
        # получаем  часть с данными
        # Находим индекс первой пустой строки, если он есть,получаем список с значениями где есть пустые строки
        empty_row_index = np.where(df_form1.isna().all(axis=1))
        if empty_row_index[0].tolist():
            row_index = empty_row_index[0][0]
            df_form1 = df_form1.iloc[:row_index]
        quantity_spec = df_form1.shape[0] // 2 # получаем количество специальностей в файле
        check_two_rows_spec = df_form1['04'].tolist() == ['01','02'] * quantity_spec # проверяем чтобы колонка 04 состояла только из 01 и 02
        if not check_two_rows_spec:
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                        'Проверьте правильность заполнения колонки 04. Для каждой спец./проф. должны быть  только строки 01 и 02 не считая строки с проверкой ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                                 columns=['Название файла', 'Строка или колонка с ошибкой',
                                                          'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue
        
        check_code_lst = df_form1['03'].tolist()  # получаем список кодов специальностей
                # Проверка на то чтобы в колонке 03 в первой строке не было пустой ячейки
        if True in mask.tolist():
            if check_code_lst[0] is np.nan or check_code_lst[0] == ' ':
                temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                    'В колонке 03 на первой строке не заполнен код специальности. ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                             columns=['Название файла', 'Строка или колонка с ошибкой',
                                                      'Описание ошибки'])
                error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
                continue
        
        # Проверка на непрерывность кода специальности, то есть на 2 строки должен быть только один код
        border_check_code = 0  # счетчик обработанных страниц
        quantity_check_code = len(check_code_lst) // 2  # получаем сколько специальностей в таблице
        flag_error_code_spec = False  # чекбокс для ошибки несоблюдения расстояния в 2 строки
        flag_error_space_spec = False  # чекбокс для ошибки заполнения кода специальности пробелом
        for i in range(quantity_check_code):
            # получаем множество отбрасывая np.nan
            temp_set = set([code_spec for code_spec in check_code_lst[border_check_code:border_check_code + 2]])
            if len(temp_set) != 1:
                flag_error_code_spec = True
            if ' ' in temp_set:
                flag_error_space_spec = True
            border_check_code += 2

        if flag_error_space_spec:
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'Обнаружены ячейки заполненные пробелом в колонке 03 !!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!!']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue

        if flag_error_code_spec:
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'ДОЛЖЕН БЫТЬ ОДИНАКОВЫЙ КОД СПЕЦИАЛЬНОСТИ НА КАЖДЫЕ 15 СТРОК (не считая строки с проверкой)!!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!!']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue


Лишняя строка
Мониторинг_трудоустройства_выпускников_форма
Не заполненная первая строка в 03
Пропущенный 01 или 02


In [572]:
df_form1.head()

,01,02,03,04,05,06,07,08,09,10,...,69,70,71,72,73,74,75,76,77,78
0,Дальневосточный федеральный округ,Республика Бурятия,15.01.05 Сварщик (электросварочные и газосваро...,01,Всего (общая численность выпускников),45,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Дальневосточный федеральный округ,Республика Бурятия,15.01.35 Мастер слесарных работ,01,Всего (общая численность выпускников),15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Дальневосточный федеральный округ,Республика Бурятия,15.01.35 Мастер слесарных работ,02,из строки 01: имеют договор о целевом обучении,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Дальневосточный федеральный округ,Республика Бурятия,15.01.33 Токарь на станках с числовым программ...,01,Всего (общая численность выпускников),18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Дальневосточный федеральный округ,Республика Бурятия,15.01.33 Токарь на станках с числовым программ...,02,из строки 01: имеют договор о целевом обучении,7,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [573]:
error_df

,Название файла,Строка или колонка с ошибкой,Описание ошибки
0,Лишняя строка,,Проверьте правильность заполнения колонки 04. ...
1,Не заполненная первая строка в 03,,В колонке 03 на первой строке не заполнен код ...
2,Пропущенный 01 или 02,,Проверьте правильность заполнения колонки 04. ...
